In [1]:
import findspark
findspark.init('D:\software\Spark\spark-3.0.0-preview2-bin-hadoop3.2') #这个args要指明SPARK_HOME 例如:findspark.init("/usr/local/spark")
from pyspark.sql import SparkSession
sc=SparkSession.builder.appName("new_spark").config("spark.executor.memory","2g").config("spark.executor.cores","4").config("spark.exector.instances","1").getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pandas as pd
sql_sc = SQLContext(sc)

In [3]:
pd_df_ratings = pd.read_csv('ratings_small.csv')
pyspark_df_ratings = sql_sc.createDataFrame(pd_df_ratings)
# print(pyspark_df_ratings.show(5, truncate=False))

In [4]:
pyspark_df_ratings = pyspark_df_ratings.drop('timestamp')
pyspark_df_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
|     1|   1129|   2.0|
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1287|   2.0|
|     1|   1293|   2.0|
|     1|   1339|   3.5|
|     1|   1343|   2.0|
|     1|   1371|   2.5|
|     1|   1405|   1.0|
|     1|   1953|   4.0|
|     1|   2105|   4.0|
|     1|   2150|   3.0|
|     1|   2193|   2.0|
|     1|   2294|   2.0|
|     1|   2455|   2.5|
|     1|   2968|   1.0|
|     1|   3671|   3.0|
+------+-------+------+
only showing top 20 rows



In [5]:
# 创建ALS模型
als = ALS(rank=3, maxIter=10, regParam=0.1, userCol='userId',itemCol='movieId', ratingCol='rating')
model = als.fit(pyspark_df_ratings)

In [6]:
# 对userId=100进行Top-N推荐
recommendations = model.recommendForAllUsers(5)
print(recommendations.where(recommendations.userId == 100).collect())

[Row(userId=100, recommendations=[Row(movieId=3879, rating=5.85002326965332), Row(movieId=83318, rating=5.22407341003418), Row(movieId=83411, rating=5.22407341003418), Row(movieId=67504, rating=5.22407341003418), Row(movieId=83359, rating=5.22407341003418)])]
